In [2]:
from __future__ import unicode_literals
import tweepy
import csv
from tweepy import Cursor
from tweepy import OAuthHandler
from tweepy import API
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
                                                                     
import pprint as pp
import pandas as pd
import time

In [3]:
consumer_key = 'HWAATgWiwSpVWYTfM2M8zoKyh'
consumer_secret = 'Qtk2aJ0kaOj3B2OWdWa0ca0fXYCrq1AJiYTRlXHHlFPOFoCaG2'
access_token = '1008796219622256641-h021XV80GH2sy7t2wocOOwzymL0ONw'
access_token_secret = '7bgHU7uUEdvNW2KpYoCiKOOR15B6ihvmkHVzVXQUinboQ'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# from requests_oauthlib import OAuth1
# 
# 
# base_twitter_url = "https://api.twitter.com/1.1/"
# 
# auth = OAuth1(client_key, client_secret, token, token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [7]:
def search_tweets (term, map_id = False, grouped = True, limit = 1):
    """
    This function uses Tweepy to fetch Twitter data
    Inputs:
        term: Search term
        map_id: Which map to merge data with
        grouped: Whether to merge with group data or not
        limit: How many 100s of search results to fetch
    Outputs:
        df: Dataframe of tweets returned from search
    """
    alltweets = []
    new_tweets = api.search(term, count = 100,tweet_mode='extended')

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    counter = 0

#     while len(new_tweets) > 0:
    if limit == False:
        print('...Fetching all tweets')
        limit = 99999
    else:
        print('..Fetching {}00 tweets'.format(limit))
        
    for i in range(limit):
        #break out if no more tweets are found
        if len(new_tweets) == 0:
            break
#         print ("...Getting tweets before %s" %(oldest))

        #all subsequent requests use the max_id param to prevent duplicates
        new_tweets = api.search(term, count = 100,max_id=oldest,tweet_mode='extended')

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        return parse_tweepy_results(alltweets)
#         print ("...%s tweets downloaded so far"%(len(alltweets)))

In [11]:
def parse_tweepy_results(alltweets, map_id = False, grouped = True):
    tweets_features = []

    for tweet in alltweets:
        tweet_dict = {}
        if str(tweet.full_text)[:3] == "RT ":
            try:
                text = tweet.retweeted_status.full_text
                retweet = True
                retweet_screen_name = tweet.retweeted_status.user.screen_name
                retweet_tweet_id = tweet.retweeted_status.id_str
                urls = []
                for u in tweet.retweeted_status.entities["urls"]:
                    urls.append(u["expanded_url"])

                hashtags = []
                for h in tweet.retweeted_status.entities["hashtags"]:
                    hashtags.append(h['text'])

                mentions = []    
                for m in tweet.retweeted_status.entities["user_mentions"]:
                    mentions.append(m['screen_name'])

            except AttributeError:
                retweet = True
                text = tweet.full_text.encode("utf-8")

        else:
            text = tweet.full_text.encode("utf-8") #indented this here
            retweet = False
            retweet_screen_name = None
            retweet_tweet_id = None

        urls = []
        for u in tweet.entities["urls"]:
            urls.append(u["expanded_url"])

        hashtags = []
        for h in tweet.entities["hashtags"]:
            hashtags.append(h['text'])

        mentions = []    
        for m in tweet.entities["user_mentions"]:
            mentions.append(m['screen_name'])

        tweet_dict["retweet_screen_name"] = retweet_screen_name
        tweet_dict["retweet_tweet_id"] = retweet_tweet_id 
        tweet_dict["user_id"] = tweet.user.id
        tweet_dict["retweet"] = retweet
        tweet_dict["text"] = text
        tweet_dict["author"] = tweet.author.name
        tweet_dict["screen_name"] = tweet.author.screen_name
        tweet_dict["tweet_id"] = tweet.id_str
        tweet_dict["time"] = tweet.created_at
        tweet_dict["mentions"] = mentions
        tweet_dict["hashtags"] = hashtags
        tweet_dict["urls"] = urls

        tweets_features.append(tweet_dict)

        #also need author id so can recreated url, might also be an easier way

    #transform the tweepy tweets into a 2D array that will populate the csv	
    # outtweets = [[tweet.author.name, tweet.id_str, tweet.created_at, tweet.full_text.encode("utf-8")] for tweet in alltweets]

    df = pd.DataFrame(tweets_features)
    
    if map_id:
        nodes = get_node_data(map_id,grouped)
        df = pd.merge(nodes,df)
        
    print('...Done!')
    return df

In [20]:
def get_full_tweets(tweet_id_list):
    print('...Getting {} tweets'.format(len(tweet_id_list)))
#     tweet_id_list = [1246082527544180736,1246086302178537473]
    result_tweets = []
    for i in range(0,len(tweet_id_list),100):
        try:
            result_tweets.extend(api.statuses_lookup(tweet_id_list[i:i+100],tweet_mode = 'extended'))
        except:
            continue
    return parse_tweepy_results(result_tweets)